In [4]:
# by Joh Schöneberg 2018

%matplotlib inline
import numpy as np
import matplotlib.pyplot as plt

from numpy import genfromtxt


import pandas as pd

from mpl_toolkits import mplot3d
#%matplotlib notebook

import math

#import cv2
import pims
from PIL import Image

import skimage

import json

#own modules / functions
import sys

sys.path.append("./modules")
import TiffProcessing as tiffproc
import InputProcessing as inproc
sys.path.append("./classes")
import Track

# input data

In [5]:
inputParameters = pd.read_csv('../../input/_inputParameters.csv',names=['key','value'])


inputDataFolder = inproc.getInputParameter(inputParameters,"inputDataFolder")
outputDataFolder = inproc.getInputParameter(inputParameters,"outputDataFolder")
ch0_outputDataFolder  = inproc.getInputParameter(inputParameters,"ch0_outputDataFolder")
ch0_trackingCsvFilename = inproc.getInputParameter(inputParameters,"ch0_trackingCsvFilename")

ch0_trackingCsvFilenameProcessed = inproc.getInputParameter(inputParameters,"ch0_trackingCsvFilenameProcessed")
ch1_outputDataFolder  = inproc.getInputParameter(inputParameters,"ch1_outputDataFolder")
ch1_trackingCsvFilename = inproc.getInputParameter(inputParameters,"ch1_trackingCsvFilename")

ch0_uniqueFilenameString = inproc.getInputParameter(inputParameters,"ch0_uniqueFilenameString")
ch1_uniqueFilenameString = inproc.getInputParameter(inputParameters,"ch1_uniqueFilenameString")


ch0_trackingProcessedBildFilename = inproc.getInputParameter(inputParameters,"ch0_trackingProcessedBildFilename")
ch1_trackingProcessedBildFilename = inproc.getInputParameter(inputParameters,"ch1_trackingProcessedBildFilename")



framerate_msec = float(inproc.getInputParameter(inputParameters,"framerate_msec"))
movieLength = int(inproc.getInputParameter(inputParameters,"movieLength"))


IndexError: index 0 is out of bounds for axis 0 with size 0

In [ ]:
ch0_filesOfInterest = inproc.getFilenamesOfInputTiffFiles(inputDataFolder,ch0_uniqueFilenameString)
ch1_filesOfInterest = inproc.getFilenamesOfInputTiffFiles(inputDataFolder,ch1_uniqueFilenameString)

In [3]:
# read the image from where the detections come from
frameId = 0

path = inputDataFolder+'/'+ch0_filesOfInterest[frameId]
print(path)
latticeMovieFrame = skimage.external.tifffile.imread(path)
latticeMovieFrame_shape = latticeMovieFrame.shape
print(latticeMovieFrame_shape)

tiffproc.maxIntensityProjection(latticeMovieFrame)

NameError: name 'ch0_filesOfInterest' is not defined

In [169]:
# read the raw csv data for further processing

path = outputDataFolder+'/'+ch0_trackingCsvFilenameProcessed
print(path)

data = pd.read_csv(path,header=0)
data.columns = ["trackId","tracklength","time[s]","frameId","x",'y',"z","A","c","lifetime","catIdx","pval_Ar"]
trackIds = data.drop_duplicates(subset='trackId')['trackId'].values

data[0:5]




/Users/johannesschoeneberg/git/LatticeTrack/data/test_input/crop_100x100x100/output/trackedFeaturesProcessed.csv


,trackId,tracklength,time[s],frameId,x,y,z,A,c,lifetime,catIdx,pval_Ar
0,1,79,0,1,7.904661e+01,6.464731e+01,8.131298e+01,2.380362e+04,5.097926e+03,398.23,8,0.0
1,1,79,5.689000e+00,2,7.850739e+01,6.542015e+01,8.129176e+01,2.002146e+04,5.046481e+03,398.23,8,0.0
2,1,79,1.137800e+01,3,7.839408e+01,6.518394e+01,8.116666e+01,1.914605e+04,4.805831e+03,398.23,8,0.0
3,1,79,1.706700e+01,4,7.802180e+01,6.560145e+01,8.168228e+01,2.065366e+04,5.580686e+03,398.23,8,0.0
4,1,79,2.275600e+01,5,7.721728e+01,6.629176e+01,8.136034e+01,1.984517e+04,4.731718e+03,398.23,8,0.0


# select subset

In [170]:
# select a subset of the tracks, e.g. for displaying only long tracks

trackIdsLong = (data[data['tracklength'] > 10]).drop_duplicates(subset='trackId')['trackId'].values
allTrackIds = np.unique(data['trackId'].values)
print('len(trackIdsLong)/len(allTrackIds)')
print(str(len(trackIdsLong))+'/'+str(len(allTrackIds)))
print()
print(trackIdsLong)
print('/')
print(allTrackIds)

len(trackIdsLong)/len(allTrackIds)
148/463

[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87  88  89  90
  91  92  93  94  95  96  97  98  99 100 101 102 103 104 105 106 107 108
 109 110 111 112 113 114 115 116 117 118 119 120 121 122 123 124 125 126
 127 128 129 130 131 132 133 134 135 136 137 138 139 140 141 142 143 144
 145 146 147 148]
/
[  1   2   3   4   5   6   7   8   9  10  11  12  13  14  15  16  17  18
  19  20  21  22  23  24  25  26  27  28  29  30  31  32  33  34  35  36
  37  38  39  40  41  42  43  44  45  46  47  48  49  50  51  52  53  54
  55  56  57  58  59  60  61  62  63  64  65  66  67  68  69  70  71  72
  73  74  75  76  77  78  79  80  81  82  83  84  85  86  87

# write individual .bild files, through the track class

In [171]:
# create Track objects for every track

import timeit


trackIdListInvestigated = trackIdsLong

start_time = timeit.default_timer()

tracks = []
cm0 = []
len0 = []
Amean0 = []
maxDist0 = []



for i in range(0,len(trackIdListInvestigated)):
    if(i %1000 == 0):
        print(str(i)+"/"+str(len(trackIdListInvestigated)))
    trackData = data[data['trackId'] == trackIdListInvestigated[i]]
    a = Track.Track(trackData,latticeMovieFrame_shape)
    tracks.append(a)
    cm0.append(a.cm)
    len0.append(a.len)
    Amean0.append(a.Amean)
    maxDist0.append(a.maxDist)
    
elapsed = timeit.default_timer() - start_time
print('time elapse: '+str(np.round(elapsed,decimals=2))+'s')

0/148
time elapse: 0.24s


In [172]:
# go through all the Tracks in the list and write a bild file

for track in tracks:
    numberOfZeros = int(np.ceil(np.log10(movieLength+1)))
    output_filename = 'track_'+str(track.id).zfill(numberOfZeros)+'.bild'
    path = outputDataFolder+'/'+output_filename
    print(path)
    track.writeBILD(path,color='orange')

/Users/johannesschoeneberg/git/LatticeTrack/data/test_input/crop_100x100x100/output/track_01.bild
.arrow 76.42115 71.40531 14.59761 nan nan nan

.arrow nan nan nan 73.38474 74.00051 15.11927

/Users/johannesschoeneberg/git/LatticeTrack/data/test_input/crop_100x100x100/output/track_02.bild
.arrow 68.50898 83.80136 39.13954 nan nan nan

.arrow nan nan nan 64.78614 86.47406 32.25419

.arrow 65.23636 80.79743 32.56751 nan nan nan

.arrow nan nan nan 64.64881 83.56337 32.17015

/Users/johannesschoeneberg/git/LatticeTrack/data/test_input/crop_100x100x100/output/track_03.bild
/Users/johannesschoeneberg/git/LatticeTrack/data/test_input/crop_100x100x100/output/track_04.bild
/Users/johannesschoeneberg/git/LatticeTrack/data/test_input/crop_100x100x100/output/track_05.bild
/Users/johannesschoeneberg/git/LatticeTrack/data/test_input/crop_100x100x100/output/track_06.bild
/Users/johannesschoeneberg/git/LatticeTrack/data/test_input/crop_100x100x100/output/track_07.bild
/Users/johannesschoeneberg/git/L

# write .bild file directly from the csv file

In [173]:
#plot them all
import math


def writeAllTracksProcessed_bildFile(trackData,trackIdListToPlot,latticeMovieFrame_shape,path,drawColor='white'):

    
    file = open(path,'w') 
    
    file.write(".transparency 0.5\n")
    file.write(".color "+drawColor+"\n")
    
    nanLineCounter = 0
    lineCounter = 0
    trackCounter = 0
    for trackID in trackIdListToPlot:
        trackCounter = trackCounter +1
        line = ".comment trackID"+str(trackID)+"\n"
        file.write(line)
        
        
        track = data[data['trackId'] == trackID]
        tracklength = track['tracklength'].iloc[0]
        
        for i in range(1,tracklength):
            tzero = track.iloc[i-1]
            tone = track.iloc[i]
            
    
            # Data for a three-dimensional line
            x0 = float(tzero['x'])
            y0 = float(tzero['y'])
            z0 = np.abs(float(tzero['z']) - latticeMovieFrame_shape[2])
            A0 = float(tzero['A'])
            
            x1 = float(tone['x'])
            y1 = float(tone['y'])
            z1 = np.abs(float(tone['z'])- latticeMovieFrame_shape[2])
            A1 = float(tone['A'])
            
            if(math.isnan(x0) or math.isnan(y0) or math.isnan(z0) or math.isnan(x1) or math.isnan(y1) or math.isnan(z1)):
                line = ".arrow "+str(x0)+" "+str(y0)+" "+str(z0)+" "+str(x1)+" "+str(y1)+" "+str(z1)+"\n" #" "+str(radius)+"\n"        
                # print(line)
                nanLineCounter = nanLineCounter +1
                continue
            
            
            #.arrow x1 y1 z1 x2 y2 z2 [r1 [r2 [rho]]] 
            
            line = ".arrow "+str(x0)+" "+str(y0)+" "+str(z0)+" "+str(x1)+" "+str(y1)+" "+str(z1)+"\n" #" "+str(radius)+"\n"        
            lineCounter = lineCounter + 1
            file.write(line)
            
    print('# tracks: '+str(trackCounter))
    print('# lines written: '+str(lineCounter))
    print('# nan lines encountered and discarded: '+str(nanLineCounter))
    print(path)
    file.close()

In [174]:
# plot all tracks

trackData = data
trackIdListToPlot = np.unique(data['trackId'].values)
latticeMovieFrame_shape = latticeMovieFrame_shape
path = outputDataFolder+'/'+ch1_trackingProcessedBildFilename
drawColor = 'white'

writeAllTracksProcessed_bildFile(trackData,trackIdListToPlot,latticeMovieFrame_shape,path,drawColor)


# tracks: 463
# lines written: 4764
# nan lines encountered and discarded: 24
/Users/johannesschoeneberg/git/LatticeTrack/data/test_input/crop_100x100x100/output/tracksProcessed.bild


In [175]:
#plot them all

#%----------------------------------------------------------------------------
#    % I. Assign category to each track
#    %----------------------------------------------------------------------------
#    % Categories:
#    % Ia)  Single tracks with valid gaps
#    % Ib)  Single tracks with invalid gaps
#    % Ic)  Single tracks cut at beginning or end
#    % Id)  Single tracks, persistent
#    % IIa) Compound tracks with valid gaps
#    % IIb) Compound tracks with invalid gaps
#    % IIc) Compound tracks cut at beginning or end
#    % IId) Compound tracks, persistent
#    
#    % The categories correspond to index 1-8, in the above order



#data = longtracks
data = data

allTracks = np.unique(data['trackId'].values)
print('# all tracks '+ str(len(allTracks)))
cat_Ia_Tracks = np.unique(data[data['catIdx'] == 1]['trackId'])
cat_Ib_Tracks = np.unique(data[data['catIdx'] == 2]['trackId'])
cat_Ic_Tracks = np.unique(data[data['catIdx'] == 3]['trackId'])
cat_Id_Tracks = np.unique(data[data['catIdx'] == 4]['trackId'])
cat_IIa_Tracks = np.unique(data[data['catIdx'] == 5]['trackId'])
cat_IIb_Tracks = np.unique(data[data['catIdx'] == 6]['trackId'])
cat_IIc_Tracks = np.unique(data[data['catIdx'] == 7]['trackId'])
cat_IId_Tracks = np.unique(data[data['catIdx'] == 8]['trackId'])



trackData = data
trackIdListToPlot = cat_Ia_Tracks
latticeMovieFrame_shape = latticeMovieFrame_shape
path = outputDataFolder+'/cat_Ia_Tracks.bild'
drawColor = 'green'
writeAllTracksProcessed_bildFile(trackData,trackIdListToPlot,latticeMovieFrame_shape,path,drawColor)

trackData = data
trackIdListToPlot = cat_Ib_Tracks
latticeMovieFrame_shape = latticeMovieFrame_shape
path = outputDataFolder+'/cat_Ib_Tracks.bild'
drawColor = 'yellow'
writeAllTracksProcessed_bildFile(trackData,trackIdListToPlot,latticeMovieFrame_shape,path,drawColor)

trackData = data
trackIdListToPlot = cat_Ic_Tracks
latticeMovieFrame_shape = latticeMovieFrame_shape
path = outputDataFolder+'/cat_Ic_Tracks.bild'
drawColor = 'sienna'
writeAllTracksProcessed_bildFile(trackData,trackIdListToPlot,latticeMovieFrame_shape,path,drawColor)

trackData = data
trackIdListToPlot = cat_Id_Tracks
latticeMovieFrame_shape = latticeMovieFrame_shape
path = outputDataFolder+'/cat_Id_Tracks.bild'
drawColor = 'red'
writeAllTracksProcessed_bildFile(trackData,trackIdListToPlot,latticeMovieFrame_shape,path,drawColor)

#-------

trackData = data
trackIdListToPlot = cat_IIa_Tracks
latticeMovieFrame_shape = latticeMovieFrame_shape
path = outputDataFolder+'/cat_IIa_Tracks.bild'
drawColor = 'cyan'
writeAllTracksProcessed_bildFile(trackData,trackIdListToPlot,latticeMovieFrame_shape,path,drawColor)

trackData = data
trackIdListToPlot = cat_IIb_Tracks
latticeMovieFrame_shape = latticeMovieFrame_shape
path = outputDataFolder+'/cat_IIb_Tracks.bild'
drawColor = 'blue'
writeAllTracksProcessed_bildFile(trackData,trackIdListToPlot,latticeMovieFrame_shape,path,drawColor)

trackData = data
trackIdListToPlot = cat_IIc_Tracks
latticeMovieFrame_shape = latticeMovieFrame_shape
path = outputDataFolder+'/cat_IIc_Tracks.bild'
drawColor = 'purple'
writeAllTracksProcessed_bildFile(trackData,trackIdListToPlot,latticeMovieFrame_shape,path,drawColor)

trackData = data
trackIdListToPlot = cat_IId_Tracks
latticeMovieFrame_shape = latticeMovieFrame_shape
path = outputDataFolder+'/cat_IId_Tracks.bild'
drawColor = 'grey'
writeAllTracksProcessed_bildFile(trackData,trackIdListToPlot,latticeMovieFrame_shape,path,drawColor)

# all tracks 463
# tracks: 118
# lines written: 400
# nan lines encountered and discarded: 0
/Users/johannesschoeneberg/git/LatticeTrack/data/test_input/crop_100x100x100/output/cat_Ia_Tracks.bild
# tracks: 146
# lines written: 1345
# nan lines encountered and discarded: 0
/Users/johannesschoeneberg/git/LatticeTrack/data/test_input/crop_100x100x100/output/cat_Ib_Tracks.bild
# tracks: 187
# lines written: 2337
# nan lines encountered and discarded: 0
/Users/johannesschoeneberg/git/LatticeTrack/data/test_input/crop_100x100x100/output/cat_Ic_Tracks.bild
# tracks: 3
# lines written: 207
# nan lines encountered and discarded: 0
/Users/johannesschoeneberg/git/LatticeTrack/data/test_input/crop_100x100x100/output/cat_Id_Tracks.bild
# tracks: 1
# lines written: 22
# nan lines encountered and discarded: 2
/Users/johannesschoeneberg/git/LatticeTrack/data/test_input/crop_100x100x100/output/cat_IIa_Tracks.bild
# tracks: 0
# lines written: 0
# nan lines encountered and discarded: 0
/Users/johannessch